# Introduction

This notebook illustrates clustering of Ti crystal orientations using data obtained from a highly deformed specimen, using EBSD.

This functionaility has been checked to run in orix-0.2.3 (May 2020). Bugs are always possible, do not trust the code blindly, and if you experience any issues please report them here: https://github.com/pyxem/orix-demos/issues

# Contents

1. <a href='#imp'> Import data</a>
2. <a href='#dis'> Compute distance matrix</a>
3. <a href='#clu'> Clustering</a>
4. <a href='#vis'> Visualisation</a>

Import orix classes and various dependencies

In [1]:
%matplotlib qt5

# Important external dependencies
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN

# orix dependencies (tested with orix 0.1.1)
from orix.quaternion.orientation import Orientation, Misorientation
from orix.quaternion.rotation import Rotation
from orix.quaternion.symmetry import D6
from orix.quaternion.orientation_region import OrientationRegion
from orix.vector.neo_euler import AxAngle
from orix.vector import Vector3d
from orix import plot

# Colorisation
from skimage.color import label2rgb
from matplotlib.colors import to_rgb, to_hex
MPL_COLORS_RGB = [to_rgb('C{}'.format(i)) for i in range(10)]
MPL_COLORS_HEX = [to_hex(c) for c in MPL_COLORS_RGB]

# Animation
import matplotlib.animation as animation

# Visualisation
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from matplotlib.lines import Line2D


plt.rc('font', size=6)

# <a id='imp'></a> 1. Import data

Load orientation mapping data specified in a standard CTF file as euler angles in degrees, following the Bunge convention.

In [2]:
filepath = './data/Ti_orientations.ctf'
dat = np.loadtxt(filepath, skiprows=1)[:, :3]

Initialize an orix Orientation object containing the data

In [3]:
ori = Orientation.from_euler(np.radians(dat))
print(ori.size)

193167


Reshape the orientation mapping data to the correct spatial dimensions for the scan

In [4]:
ori = ori.reshape(381,507)

Selct a subset of the data to reduce compute time

In [5]:
ori = ori[-100:,:200]
print(ori.size)

20000


Define the fundamental region based on the D6 symmetry of Ti

In [6]:
fundamental_region = OrientationRegion.from_symmetry(D6)

# <a id='dis'></a> 2. Compute distance matrix

Clustering algorithms require a distance matrix, $D_{ij}$, containing the distance, $d(o_i, o_j)$, between all (mis)orientations to be computed. We define this distance as the minimum rotational angle relating (mis)orientations amongst all symmetry equivalent rotations.

Computation of the distance matrix is the most computationally intensive part of this data processing. Here we provide 3 alternative implementations that use resources differently:

1. Calculate the outer products needed to determine the distance metric and compute the associated angle. Minimise with respect to the tensor axes corresponding to symmetry. Uses lots of of computer memory.


2. Iterate over each factor updating the distance as it gets smaller. Uses little computer memory but computation takes a long time.


3. Iterating over pairs of data points while performing an outer product minimisation with respect to the symmetry elements for each pair. Uses more computer memory to avoid excessive computation times.


**WARNING: The computation in Section 2.1 takes time and may exceed limits on your machine. This section is commented out to avoid unintentional use. A pre-computed solution is provided in Section 2.2!**

## 2.1. Compute the distance matrix yourself (optional)

**Option 1: high (ca. 32 Gb) RAM, fast vectorized computation**

Computes every possibility in a single tensor, then minimises.

In [ ]:
#misori_equiv = D6.outer(~ori).outer(ori).outer(D6)
#D = misori_equiv.angle.data.min(axis=(0, 2))

**Option 2: low RAM, slow iteration**

Iterates through every pair of orientations.

In [ ]:
#D = np.empty(ori.shape + ori.shape)
#
#for i, j in tqdm_notebook(list(icombinations(range(ori.size), 2))):
#    idx_1, idx_2 = np.unravel_index(i, ori.shape), np.unravel_index(i, ori.shape)
#    o_1, o_2 = ori[idx_1], ori[idx_2]
#    misori = D6.outer(~o_1).outer(o_2).outer(D6)
#    d = misori.angle.data.min(axis=(0, 3))
#    D[idx_1[0], idx_1[1], idx_2[0], idx_2[1]] = d
#    D[idx_2[0], idx_2[1], idx_1[0], idx_1[1]] = d

**Option 3: RAM vs. speed compromise**

Precomputes one set of equivalent orientations.

In [ ]:
#D = np.zeros(ori.shape + ori.shape)
#D.fill(np.infty)
#
#OS2 = ori.outer(D6)
#
#for i in tqdm_notebook(range(ori.size)):
#    idx = np.unravel_index(i, ori.shape)
#    misori = D6.outer(~ori[idx]).outer(OS2)
#    d = misori.angle.data.min(axis=(0, -1))
#    D[idx[0], idx[1], ...] = np.minimum(D[idx[0], idx[1], ...], d)

## 2.2. Load a distance matrix we computed for you

Load the precomputed distance matrix for the data subset

In [7]:
filepath = './data/ori-distance((100, 200)).npy'
D = np.load(filepath)

# <a id='clu'></a> 3. Clustering

Perform clustering

In [8]:
dbscan = DBSCAN(0.1, 40, metric='precomputed').fit(D.reshape(ori.size, ori.size))
print('Labels:', np.unique(dbscan.labels_))
labels = dbscan.labels_.reshape(ori.shape)
n_clusters = len(np.unique(dbscan.labels_)) - 1
print('Number of clusters:', n_clusters)

Labels: [-1  0  1  2  3  4  5]
Number of clusters: 6


Calculate the mean orientation for each cluster

In [9]:
cluster_means = Orientation.stack([ori[labels == label].mean() for label in np.unique(dbscan.labels_)[1:]]).flatten()
cluster_means = cluster_means.set_symmetry(D6)

Inspect rotation axes in the axis-angle representation

In [10]:
cluster_means_axangle = AxAngle.from_rotation(cluster_means)

Recenter data relative to the matrix cluster and recompute means

In [11]:
ori_recentered = (~cluster_means[0]) * ori
ori_recentered = ori_recentered.set_symmetry(D6)
cluster_means_recentered = Orientation.stack([ori_recentered[labels == label].mean() for label in np.unique(dbscan.labels_)[1:]]).flatten()
cluster_means_axangle = AxAngle.from_rotation(cluster_means_recentered)

Inspect recentered rotation axes in the axis-angle representation

In [12]:
cluster_means_recentered.axis

Vector3d (6,)
[[ 0.      0.      1.    ]
 [-0.9011 -0.3965 -0.1756]
 [ 0.0856 -0.8516  0.5172]
 [-0.5222  0.2419 -0.8178]
 [-0.8673  0.4797 -0.133 ]
 [ 0.3198 -0.6549  0.6847]]

# <a id='vis'></a> 4. Visualisation

Specify colours and lines to identify each cluster

In [13]:
# get label colors
colors = [to_rgb('C{}'.format(i)) for i in range(10)]
labels_rgb = label2rgb(labels, colors=colors)

# Create map and lines pointing to cluster means
mapping = labels_rgb
collection = Line3DCollection([((0, 0, 0), tuple(cm)) for cm in cluster_means_axangle.data], colors=colors)

Plot the orientation clusters within the fundamental zone for D6 symmetry Ti

In [14]:
fig = plt.figure(figsize=(3.484252, 3.484252))
gridspec = plt.GridSpec(1, 1, left=0, right=1, bottom=0, top=1, hspace=0.05)

ax_ori = fig.add_subplot(gridspec[0], projection='axangle', proj_type='ortho')
ax_ori.scatter(ori_recentered, c=labels_rgb.reshape(-1, 3), s=1)
ax_ori.plot_wireframe(fundamental_region, color='black', linewidth=0.5, alpha=0.1, rcount=181, ccount=361)
ax_ori.add_collection3d(collection)

ax_ori.set_axis_off()
ax_ori.set_xlim(-1, 1)
ax_ori.set_ylim(-1, 1)
ax_ori.set_zlim(-1, 1)
ax_ori.view_init(90, -30)

handles = [
    Line2D(
        [0], [0], 
        marker='o', color='none', 
        label=i+1, 
        markerfacecolor=color, markersize=5
    ) for i, color in enumerate(colors[:n_clusters])
]

ax_ori.legend(handles=handles, loc='lower right', ncol=2, numpoints=1, labelspacing=0.15, columnspacing=0.15, handletextpad=0.05)

/home/hremadmin/anaconda3/envs/orix/lib/python3.8/site-packages/orix-0.2.3.dev0-py3.8.egg/orix/quaternion/orientation_region.py:201: RuntimeWarning: divide by zero encountered in reciprocal


/home/hremadmin/anaconda3/envs/orix/lib/python3.8/site-packages/mpl_toolkits/mplot3d/proj3d.py:61: RuntimeWarning: invalid value encountered in double_scalars
  u = (x01*x21 + y01*y21) / (x21**2 + y21**2)


Plot side view of orientation clusters in the fundamental zone for D6 symmetry Ti

In [ ]:
plt.close('all')
fig = plt.figure(figsize=(3.484252 * 2, 1.5 * 2))
gridspec = plt.GridSpec(1, 1, left=0, right=1, bottom=0, top=1, hspace=0.05)

ax_ori = fig.add_subplot(gridspec[0], projection='axangle', proj_type='ortho', aspect='equal')
ax_ori.scatter(ori_recentered, c=labels_rgb.reshape(-1, 3), s=1)
ax_ori.plot_wireframe(fundamental_region, color='black', linewidth=0.5, alpha=0.1, rcount=181, ccount=361)
# ax_ori.add_collection3d(collection)

ax_ori.set_axis_off()
ax_ori.set_xlim(-1, 1)
ax_ori.set_ylim(-1, 1)
ax_ori.set_zlim(-1, 1)
ax_ori.view_init(0, -30)

Plot map indicating spatial locations associated with each cluster

In [ ]:
plt.close('all')
map_ax = plt.axes()
map_ax.imshow(mapping)

map_ax.set_xticks([])
map_ax.set_yticks([])